# Hosting models on Grid

Grid offers both: Machine Learning as a Service and Encrypted Machine Learning as a service. In this series of tutorials we show how you can serve and query models on Grid.

## 3. Host and query an encrypted model

In the previous tutorial we served a CNN for classifying images with different 2 types of skin deseases: benign keratosis and melanoma (type of skin cancer). In this tutorial we show how to serve this model on a **encrypted way** on Grid.

### Imports and model specifications

In [1]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Import dependencies
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import torch as th
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import syft as sy
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import torch.nn as nn
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import torch.nn.functional as F
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import grid as gr
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                import helper

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Hook
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                hook = sy.TorchHook(th)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                me = hook.local_worker
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                me.is_client_worker = False

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Connect to nodes
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                alice.connect()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                bob.connect()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                charlie.connect()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                dan.connect()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # Connect nodes to each other
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                gr.connect_all_nodes([bob, alice, charlie, dan])

### Load dataset

In [4]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

### Making a model ready to be served and encrypted

In order to serve the model it needs to be serializable. A Plan is intended to store a sequence of torch operations, just like a function, but it allows to send this sequence of operations to remote workers and to keep a reference to it. You can learn more about plans in [Syft's tutorials](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2008%20-%20Introduction%20to%20Plans.ipynb).


### Define Model

Let's load the model we just trained.

In [6]:
model = helper.make_model(is_plan=True)
model.load_state_dict(th.load("binary-skin-cancer-detection-model"))
model.eval()

<Net Net id:convnet owner:me>

In [7]:
model.build(data)

In [8]:
helper.test(model, test_generator)

Average loss: 0.6244, Accuracy: 751/1106 (67.9%)



(0.6243579933322144, 67.90235081374323)

## Host model

In [9]:
alice.host_model(model)

<Net Net id:convnet owner:me location:Alice built>

In [ ]:
from IPython.display import display_html

def restart_kernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
restart_kernel()

In [1]:
# Import dependencies
import torch as th
import syft as sy
import torch.nn as nn
import torch.nn.functional as F
import grid as gr
import helper

# Hook
hook = sy.TorchHook(th)
me = hook.local_worker
me.is_client_worker = False
    
# Connect to nodes
alice = gr.WebsocketGridClient(hook, "http://localhost:3001", id="Alice")
alice.connect()
bob = gr.WebsocketGridClient(hook, "http://localhost:3000", id="Bob")
charlie = gr.WebsocketGridClient(hook, "http://localhost:3002", id="James")
dan = gr.WebsocketGridClient(hook, "http://localhost:3003", id="Dan")
bob.connect()
charlie.connect()
dan.connect()

# Connect nodes to each other
gr.connect_all_nodes([bob, alice, charlie, dan])

### Load dataset

In [2]:
df = helper.read_skin_cancer_dataset()
train_df, valid_df, test_df = helper.split_data(df)

# These values are from Part 1.
input_size = 32
train_mean, train_std = (th.tensor([0.6979, 0.5445, 0.5735]), th.tensor([0.0959, 0.1187, 0.1365]))

# Create a test dataloader
test_set = helper.Dataset(test_df, transform=helper.transform(input_size, train_mean, train_std))
test_generator = th.utils.data.DataLoader(test_set, batch_size=1, shuffle=True)

# Get a data sample and a target
data, target = next(iter(test_generator))

In [3]:
x_ptr = data.fix_precision().share(bob, charlie, crypto_provider=dan)

### Get a copy of the private model

In [4]:
private_model = alice.get_private_model("convnet", workers=[bob, charlie], crypto_provider=dan)

### Run encrypted inference

In [5]:
%%time
print(private_model(x_ptr).get().float_prec())

KeyError: 'Object "56916464533" not found on worker!!!You just tried to interact with an object ID:56916464533 on <VirtualWorker id:me #objects:64> which does not exist!!! Use .send() and .get() on all your tensors to make sure they\'reon the same machines. If you think this tensor does exist, check the ._objects dictionaryon the worker and see for yourself!!! The most common reason this error happens is because someone calls.get() on the object\'s pointer without realizing it (which deletes the remote object and sends it to the pointer). Check your code to make sure you haven\'t already called .get() on this pointer!!!'

In [7]:
me._objects.keys()

dict_keys([29100689974, 18920352529, 16770090523, 20175647788, 88663703297, 53403450792, 52277785984, 6460688328, 40699742492, 36979113274, 9564373667, 71052245454, 28519877557, 57827451351, 55187692273, 12851169269, 92078711809, 66937121806, 6659350017, 42972136631, 38849641316, 52107766429, 59923361239, 30509229623, 15637689678, 12282074847, 16881363934, 98258782914, 69791478251, 53572762624, 79630605919, 57880109791, 66854311629, 44446285595, 5689565620, 52758966697, 30350309884, 31651896562, 51904794816, 44092261221, 956538253, 81411599079, 24571081680, 97727158480, 13689645002, 99883270808, 1170602582, 87553082263, 49751707647, 6056924283, 22038296744, 1605563435, 18932820698, 59448256002, 58295292669, 42113989654, 25108411585, 14136622363, 76615658003, 98233979594, 60734197148, 89936717966, 14861072075, 15824550441])

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

## Star PySyft on GitHub
The easiest way to help our community is just by starring the GitHub repos! This helps raise awareness of the cool tools we're building.

[Star PySyft](https://github.com/OpenMined/PySyft)

## Join our Slack!
The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at http://slack.openmined.org

## Join a Code Project!
The best way to contribute to our community is to become a code contributor! At any time you can go to PySyft GitHub Issues page and filter for "Projects". This will show you all the top level Tickets giving an overview of what projects you can join! If you don't want to join a project, but you would like to do a bit of coding, you can also look for more "one off" mini-projects by searching for GitHub issues marked "good first issue".

[PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
[Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

## Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)